In [2]:
import numpy as np
import pandas as pd

# import nltk
# # nltk.download('stopwords')
# # nltk.download('omw-1.4')
# # nltk.download('wordnet')
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.calibration
import sklearn.preprocessing
import matplotlib.pyplot as plt

import tabulate

# READ DATA & CLEANING - metadata_team
### 1. READ DATA
### 2. Generate Yearly Window & 2 Years Window & 5 Years Window
##### 2.1. CALCULATE YEARLY WINDOW (SUMMATION OF ALL CITATIONS)
##### 2.2. CALCULATE 2 YEARS WINDOW (PUBLISH YEAR - PUBLISH YEAR + 2)
##### 2.3. CALCULATE 5 YEARS WINDOW (PUBLISH YEAR - PUBLISH YEAR + 5)
### 3. DROP 'NAN'S IN THE COLUMNS ['PY'],['UNIQUE NATIONALITIES']
##### 3.1 SELECT DATA RELATED TO SDG 7 - ENERGY & SEPARATE THE UNIQUE NATIONALITIES
##### 3.2 NULL CHECK

In [3]:
data = pd.read_pickle('data/metadata_team.pkl')
data.head(4)

,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,...,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,Nationalities,Unique Nationalities
0,J,"Sztumski, W",NaN,NaN,NaN,"Sztumski, Wieslaw",NaN,NaN,The Mythology of Sustainable Development,PROBLEMY EKOROZWOJU,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,[[ ]],[[ ]]
1,J,"Degai, TS; Petrov, AN",NaN,NaN,NaN,"Degai, Tatiana S.; Petrov, Andrey N.",NaN,NaN,Rethinking Arctic sustainable development agen...,INTERNATIONAL JOURNAL OF SUSTAINABLE DEVELOPME...,...,0,0,0,0,0,0,0,1,"[Russia, Russia, USA, Russia]","[Russia, USA]"
2,S,"Kanapathy, S; Lee, KE; Mokhtar, M; Sivapalan, ...",NaN,"Lee, KE",NaN,"Kanapathy, Suganty; Lee, Khai Ern; Mokhtar, Ma...",NaN,NaN,Enculturing Sustainable Development Concept Th...,CONCEPTS AND APPROACHES FOR SUSTAINABILITY MAN...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"[Malaysia, Malaysia, Malaysia, Malaysia]",[Malaysia]
3,J,"Selvi, M; Selvi, M",NaN,NaN,NaN,"Selvi, M; Selvi, M",NaN,NaN,Environmental education for sustainable develo...,FRESENIUS ENVIRONMENTAL BULLETIN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[Turkey],[Turkey]


In [4]:
countries = list(np.unique(data['Unique Nationalities'].tolist))
countries

[<bound method IndexOpsMixin.tolist of 0                   [[ ]]
 1           [Russia, USA]
 2              [Malaysia]
 3                [Turkey]
 4               [Jamaica]
                ...       
 87787           [Croatia]
 87788    [Australia, USA]
 87789               [USA]
 87790            [Brazil]
 87791             [Italy]
 Name: Unique Nationalities, Length: 87792, dtype: object>]

In [5]:
mydata = data[data.PY >= 1901.0]
mydata = mydata.reset_index(drop=True)
mydata

,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,...,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,Nationalities,Unique Nationalities
0,J,"Sztumski, W",NaN,NaN,NaN,"Sztumski, Wieslaw",NaN,NaN,The Mythology of Sustainable Development,PROBLEMY EKOROZWOJU,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,[[ ]],[[ ]]
1,J,"Degai, TS; Petrov, AN",NaN,NaN,NaN,"Degai, Tatiana S.; Petrov, Andrey N.",NaN,NaN,Rethinking Arctic sustainable development agen...,INTERNATIONAL JOURNAL OF SUSTAINABLE DEVELOPME...,...,0,0,0,0,0,0,0,1,"[Russia, Russia, USA, Russia]","[Russia, USA]"
2,S,"Kanapathy, S; Lee, KE; Mokhtar, M; Sivapalan, ...",NaN,"Lee, KE",NaN,"Kanapathy, Suganty; Lee, Khai Ern; Mokhtar, Ma...",NaN,NaN,Enculturing Sustainable Development Concept Th...,CONCEPTS AND APPROACHES FOR SUSTAINABILITY MAN...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"[Malaysia, Malaysia, Malaysia, Malaysia]",[Malaysia]
3,J,"Selvi, M; Selvi, M",NaN,NaN,NaN,"Selvi, M; Selvi, M",NaN,NaN,Environmental education for sustainable develo...,FRESENIUS ENVIRONMENTAL BULLETIN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[Turkey],[Turkey]
4,J,"Ferguson, T; Roofe, C; Cook, LD",NaN,NaN,NaN,"Ferguson, Therese; Roofe, Carmel; Cook, Lorain...",NaN,NaN,Teachers' perspectives on sustainable developm...,ENVIRONMENTAL EDUCATION RESEARCH,...,0,0,0,0,0,0,0,0,"[Jamaica, Jamaica]",[Jamaica]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86300,J,"Matic, S; Delac, D",NaN,NaN,NaN,"Matic, Slavko; Delac, Damir",NaN,NaN,SILVICULTURAL TREATMENTS AS A METHOD OF INCREA...,SUMARSKI LIST,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[Croatia, Croatia, Croatia]",[Croatia]
86301,B,"Bawden, R",NaN,"Campbell, J; Baikaloff, N; Power, C",NaN,"Bawden, Richard",NaN,NaN,EDUCATING FOR CAPABILITY FOR INCLUSIVE WELL-BEING,TOWARDS A GLOBAL COMMUNITY: EDUCATING FOR TOMO...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[Australia, USA, Australia, Australia]","[Australia, USA]"
86302,J,"Eckenwiler, L",NaN,NaN,NaN,"Eckenwiler, Lisa",NaN,NaN,Displacement and solidarity: An ethic of place...,BIOETHICS,...,0.0,0.0,0.0,0.0,0.0,3.0,5.0,8.0,"[USA, USA]",[USA]
86303,J,"Nobre, AL",NaN,NaN,NaN,"Nobre, Ana Luiza",NaN,NaN,A City at Play: Rio de Janeiro on the Eve of t...,ARCHITECTURAL DESIGN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[Brazil, Brazil, Brazil]",[Brazil]


### 2. Generate Yearly Window & 2 Years Window & 5 Years Window

In [6]:
df_cols = np.array(mydata.columns.to_list())
ind_yr1 = np.where(df_cols == '1901.0')[0][0]
ind_yrlast = np.where(df_cols == '2021.0')[0][0]

yearlycitation = np.sum(mydata.iloc[:,ind_yr1:ind_yrlast], axis = 1)

In [7]:
publishyear = mydata['PY']
twoyearwindow = mydata['PY'] + 2
# fiveyearwindow = mydata['PY'] + 5

In [8]:
twoyearcitation = []
# fiveyearcitation = []

In [9]:
for i in range (len(mydata)):
    if twoyearwindow[i] >= 2019.0:
        twoyearwindow[i] = 2021
# for i in range (len(mydata)):
#     if fiveyearwindow[i] >= 2016.0:
#         fiveyearwindow[i] = 2021

In [10]:
for i in range (len(mydata)):
    temp = []
    start = int(publishyear[i])
    end = int(twoyearwindow[i])
    for j in range (start, end+1):
        temp.append(mydata[j][i])
    twoyearcitation.append(sum(temp))

In [11]:
# for i in range (len(mydata)):
#     temp = []
#     start = int(publishyear[i])
#     end = int(fiveyearwindow[i])
#     for j in range (start, end+1):
#         temp.append(mydata[j][i])
#     fiveyearcitation.append(sum(temp))

In [12]:
mydata['total_citations'] = yearlycitation
mydata['2 years window'] = twoyearcitation
# mydata['5 years window'] = fiveyearcitation

In [13]:
mydata.head()

,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,...,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,Nationalities,Unique Nationalities,total_citations,2 years window
0,J,"Sztumski, W",NaN,NaN,NaN,"Sztumski, Wieslaw",NaN,NaN,The Mythology of Sustainable Development,PROBLEMY EKOROZWOJU,...,0.0,0.0,0.0,1.0,0.0,0.0,[[ ]],[[ ]],10.0,6.0
1,J,"Degai, TS; Petrov, AN",NaN,NaN,NaN,"Degai, Tatiana S.; Petrov, Andrey N.",NaN,NaN,Rethinking Arctic sustainable development agen...,INTERNATIONAL JOURNAL OF SUSTAINABLE DEVELOPME...,...,0,0,0,0,0,1,"[Russia, Russia, USA, Russia]","[Russia, USA]",0.0,1.0
2,S,"Kanapathy, S; Lee, KE; Mokhtar, M; Sivapalan, ...",NaN,"Lee, KE",NaN,"Kanapathy, Suganty; Lee, Khai Ern; Mokhtar, Ma...",NaN,NaN,Enculturing Sustainable Development Concept Th...,CONCEPTS AND APPROACHES FOR SUSTAINABILITY MAN...,...,0.0,0.0,0.0,0.0,0.0,1.0,"[Malaysia, Malaysia, Malaysia, Malaysia]",[Malaysia],0.0,1.0
3,J,"Selvi, M; Selvi, M",NaN,NaN,NaN,"Selvi, M; Selvi, M",NaN,NaN,Environmental education for sustainable develo...,FRESENIUS ENVIRONMENTAL BULLETIN,...,0.0,0.0,0.0,0.0,0.0,0.0,[Turkey],[Turkey],0.0,0.0
4,J,"Ferguson, T; Roofe, C; Cook, LD",NaN,NaN,NaN,"Ferguson, Therese; Roofe, Carmel; Cook, Lorain...",NaN,NaN,Teachers' perspectives on sustainable developm...,ENVIRONMENTAL EDUCATION RESEARCH,...,0,0,0,0,0,0,"[Jamaica, Jamaica]",[Jamaica],0.0,0.0


### 3. DROP 'NAN'S IN THE COLUMNS ['PY'],['UNIQUE NATIONALITIES'][

### Drop zeros in citation columns

In [14]:
mydata['Unique Nationalities'] = mydata['Unique Nationalities'].astype(str) 
mydata = mydata[mydata['Unique Nationalities'].str.contains("[[' ']]", regex=False) == False]

# removing rows where AB is nan/null
mydata = mydata.reset_index()
# null_ind = np.where(mydata["AB"].isna())[0].tolist()
# # len(null_ind) #1021
# mydata = mydata.drop(index=null_ind) #drop by index

# null_ind_cit = np.where(mydata["2 years window"].isna())[0].tolist()
# # len(null_ind) #1021
# mydata = mydata.drop(index=null_ind_cit) #drop by index

#dropping rows where 2 years window is 0.0
mydata = mydata[mydata["2 years window"] != 0]

#dropping rows where total citations is 0.0
mydata = mydata[mydata["total_citations"] != 0]

mydata['PY'] = mydata['PY'].astype(int)

mydata = mydata[mydata['AB'].notna()]

mydata['AB'] = mydata['AB'].str.lower()

mydata['TI'] = mydata['TI'].str.lower()

mydata['DE'] = mydata['DE'].str.lower()

mydata

,index,PT,AU,BA,BE,GP,AF,BF,CA,TI,...,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,Nationalities,Unique Nationalities,total_citations,2 years window
4,5,J,"Agbedahin, AV",NaN,NaN,NaN,"Agbedahin, Adesuwa Vanessa",NaN,NaN,"sustainable development, education for sustain...",...,0.0,0.0,0.0,4.0,11.0,10.0,"[Africa, Africa]",['Africa'],15.0,25.0
7,8,J,"Holden, E; Linnerud, K; Banister, D",NaN,NaN,NaN,"Holden, Erling; Linnerud, Kristin; Banister, D...",NaN,NaN,the imperatives of sustainable development,...,0.0,5.0,30.0,27.0,37.0,15.0,"[England, Norway, Norway, England]",['England' 'Norway'],99.0,114.0
8,9,J,"Nousheen, A; Zai, SAY; Waseem, M; Khan, SA",NaN,NaN,NaN,"Nousheen, Ayesha; Zai, Sajid Ali Yousuf; Wasee...",NaN,NaN,education for sustainable development (esd): e...,...,0.0,0.0,0.0,0.0,6.0,13.0,"[Pakistan, Pakistan, Pakistan, Pakistan]",['Pakistan'],6.0,19.0
9,10,S,"Garcia, J; da Silva, SA; Carvalho, AS; Guerra,...",NaN,"Davim, JP",NaN,"Garcia, Jessica; da Silva, Sthefanie Aguiar; C...",NaN,NaN,education for sustainable development and its ...,...,0.0,0.0,1.0,3.0,1.0,6.0,"[Brazil, Brazil]",['Brazil'],5.0,11.0
11,12,J,"Moffatt, I",NaN,NaN,NaN,"Moffatt, I.",NaN,NaN,on measuring sustainable development indicators,...,1.0,1.0,0.0,1.0,1.0,1.0,"[Scotland, Scotland]",['Scotland'],23.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82294,86286,J,"Jimenez, JM; Cervantes, CR; Lopez, MH",NaN,NaN,NaN,"Mendoza Jimenez, Javier; Roman Cervantes, Cand...",NaN,NaN,social economy and public procurement: the res...,...,0.0,0.0,0.0,0.0,2.0,1.0,"[Spain, Spain]",['Spain'],2.0,3.0
82300,86292,J,"Ezran, C; Bonds, MH; Miller, AC; Cordier, LF; ...",NaN,NaN,NaN,"Ezran, Camille; Bonds, Matthew H.; Miller, Ann...",NaN,NaN,assessing trends in the content of maternal an...,...,0.0,0.0,0.0,0.0,2.0,3.0,"[France, USA, Madagascar, USA, Madagascar, Fra...",['France' 'Madagascar' 'USA'],2.0,5.0
82301,86293,J,"Singh, GG; Lerner, J; Mach, M; Murray, CC; Ran...",NaN,NaN,NaN,"Singh, Gerald G.; Lerner, Jackie; Mach, Megan;...",NaN,NaN,scientific shortcomings in environmental impac...,...,0.0,0.0,0.0,0.0,2.0,4.0,"[Canada, Canada, Canada, USA, Canada, Canada, ...",['Canada' 'USA'],2.0,6.0
82310,86302,J,"Eckenwiler, L",NaN,NaN,NaN,"Eckenwiler, Lisa",NaN,NaN,displacement and solidarity: an ethic of place...,...,0.0,0.0,0.0,3.0,5.0,8.0,"[USA, USA]",['USA'],8.0,16.0


##### 3.1 SELECT DATA RELATED TO SDG 7 - ENERGY & SEPARATE THE UNIQUE NATIONALITIES

In [15]:
sdg7 = "energy|electricity" #because sdg7

only_AB = mydata[mydata['AB'].notna() | mydata['DE'].notna() | mydata['TI'].notna()] #drop NANs in DE column to be able to do string match (n went from 80672 -> 70121)
only_AB['AB'] = only_AB['AB'].str.lower()
only_AB['TI'] = only_AB['TI'].str.lower()
only_AB['DE'] = only_AB['DE'].str.lower()

df = only_AB[only_AB['DE'].str.contains(sdg7, na = False) | only_AB['AB'].str.contains(sdg7, na = False) | only_AB['TI'].str.contains(sdg7, na = False)]

df = df[df.PY != 2021]
# df_bulk = df[['PY', 'Unique Nationalities', 'total_citations']]
df.head(2)
#count age till 2021 because total citations were counted till 2021
sdg7 = df[['PY', 'Unique Nationalities', '2 years window', 'total_citations']]
#---------------------------------------------------------------------------------------------------------------------
# sdg7_index = df.groupby(['PY','Unique Nationalities'], as_index=False)['2 years window'].sum()#.mean()#.sum()
# # sdg3_catchall

# # sdg7 = sdg7[~sdg7['PY'].isin(['2019','2020', '2021'])] #these don't have many citations when latest papers
# sdg7_index = sdg7_index[sdg7_index.PY != 2021]
# # sdg7 = sdg7[sdg7.PY != 2020]
# # sdg7 = sdg7[sdg7.PY != 2019]
# sdg7_index

In [90]:
# AR Index for Bulk Knowledge

py_groups_bulk = sdg7[['PY', 'Unique Nationalities', 'total_citations']]
py_groups_bulk = py_groups_bulk.groupby('PY')

for py_name, py_group in py_groups_bulk:
    py_group = py_group.sort_values("total_citations", ascending=False)
    py_group = py_group.reset_index(drop=True)
#     print(py_group)
        
    ar_index = 0
    h = 1
    for index, row in py_group.iterrows():
        if (row['total_citations'] >= h) and (row['PY'] < 2021):
            ar_index += row['total_citations'] / (2021 - row['PY'])
        h += 1
    sdg7.loc[(sdg7['PY']  == py_group['PY'][0]), 'ar index bulk'] = np.sqrt(ar_index)
    print(py_group['PY'][0], np.sqrt(ar_index))

1991 0.8755950357709131
1992 0.454858826147342
1993 0.823754471047914
1994 1.1221672153735642
1995 1.8501559185854914
1996 2.734958866235469
1997 3.2015621187164243
1998 4.313779740804335
1999 6.39246715780663
2000 11.032419757890185
2001 7.981227975693966
2002 14.339859795247945
2003 12.418624006798106
2004 12.083045973594572
2005 14.730919862656235
2006 19.894722918402255
2007 21.20646801601543
2008 20.17042770607887
2009 25.406692031825006
2010 27.396084657217983
2011 28.719331468542226
2012 26.820390253188584
2013 26.66692708206178
2014 29.99047467828029
2015 32.11437891869207
2016 32.32645975048923
2017 37.986839826445156
2018 40.25750447639131
2019 36.09709129556009
2020 21.470910553583888


/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/195603419.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdg7.loc[(sdg7['PY']  == py_group['PY'][0]), 'ar index bulk'] = np.sqrt(ar_index)


In [94]:
print("Number of rows extracted for SDG7:", len(sdg7), "/", len(mydata))
print("Percentage of rows extracted for SDG7:", (len(sdg7) / len(mydata)) * 100, "%")

Number of rows extracted for SDG7: 7538 / 36934
Percentage of rows extracted for SDG7: 20.409378892077758 %


In [95]:
sdg7 #2384x5
sdg7 = sdg7.drop(['Unique Nationalities'], axis=1)

In [96]:
sdg7_new = sdg7.groupby(['PY'], as_index=False).agg({'2 years window': 'sum', 'ar index bulk': 'mean'})
sdg7_new

,PY,2 years window,ar index bulk
0,1991,6.0,0.875595
1,1992,2.0,0.454859
2,1993,4.0,0.823754
3,1994,6.0,1.122167
4,1995,3.0,1.850156
5,1996,13.0,2.734959
6,1997,19.0,3.201562
7,1998,29.0,4.313780
8,1999,46.0,6.392467
9,2000,62.0,11.032420


### Reading WDI data

In [97]:
#read in WDI data
only_wdi = pd.read_csv('data/WDI_onlydata.csv')
wdi_series = pd.read_csv('data/WDI_series.csv')

#get data for only energy indicators
eg_wdi = only_wdi[(only_wdi['Indicator Code'].str.startswith("EG."))]
eg_indicators = eg_wdi['Indicator Name'].unique().tolist() #all the energy indicators

#abstarct data only has "energy|electricity" ABs from 1996, so we'll remove the data in wdi indicators before that (1960-1996)
eg_wdi = eg_wdi.drop(eg_wdi.iloc[:, 4:40],axis = 1)
# eg_wdi = eg_wdi.drop(eg_wdi.iloc[:, -1],axis = 1) #removing 2019-2020, because very small num of citations

#convert country name to lower case to match ab data frame
eg_wdi['Country Name'] = eg_wdi['Country Name'].str.lower()

##################################################################
eg_wdi.head(4)

,Country Name,Country Code,Indicator Name,Indicator Code,1996,1997,1998,1999,2000,2001,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,africa eastern and southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,11.435351,11.805706,...,16.559819,16.936004,17.337896,17.687092,18.140971,18.491344,18.825520,19.272212,19.628009,NaN
1,africa eastern and southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,NaN,NaN,NaN,NaN,3.500037,3.692771,...,6.281667,6.499471,6.680066,6.859110,7.016238,7.180364,7.322294,7.517191,7.651598,NaN
2,africa eastern and southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,NaN,NaN,NaN,NaN,32.330196,32.698002,...,37.601816,37.855399,38.046781,38.326255,38.468426,38.670044,38.722783,38.927016,39.042839,NaN
3,africa eastern and southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,19.811283,19.900569,...,31.844384,31.794160,32.001027,33.871910,38.880173,40.261358,43.061877,44.270860,45.803485,NaN


In [98]:
eg_indicators

['Access to clean fuels and technologies for cooking (% of population)',
 'Access to clean fuels and technologies for cooking, rural (% of rural population)',
 'Access to clean fuels and technologies for cooking, urban (% of urban population)',
 'Access to electricity (% of population)',
 'Access to electricity, rural (% of rural population)',
 'Access to electricity, urban (% of urban population)',
 'Alternative and nuclear energy (% of total energy use)',
 'Combustible renewables and waste (% of total energy)',
 'Electric power consumption (kWh per capita)',
 'Electric power transmission and distribution losses (% of output)',
 'Electricity production from coal sources (% of total)',
 'Electricity production from hydroelectric sources (% of total)',
 'Electricity production from natural gas sources (% of total)',
 'Electricity production from nuclear sources (% of total)',
 'Electricity production from oil sources (% of total)',
 'Electricity production from oil, gas and coal sources

##### 1.2 PUT ALL INDICATOR DATA INTO ONE DATAFRAME

In [99]:
Indicators = pd.DataFrame()

In [100]:
temp = eg_indicators[0]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
cooking_data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'cooking'])
cooking_data #this is tidy!

,Country Name,Year,cooking
0,africa eastern and southern,1996,NaN
1,africa eastern and southern,1997,NaN
2,africa eastern and southern,1998,NaN
3,africa eastern and southern,1999,NaN
4,africa eastern and southern,2000,11.435351
...,...,...,...
6645,zimbabwe,2016,29.800000
6646,zimbabwe,2017,29.800000
6647,zimbabwe,2018,29.900000
6648,zimbabwe,2019,30.100000


In [101]:
temp = eg_indicators[1]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
rural_cooking_data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'rural_cooking'])
rural_cooking_data #this is tidy!

,Country Name,Year,rural_cooking
0,africa eastern and southern,1996,NaN
1,africa eastern and southern,1997,NaN
2,africa eastern and southern,1998,NaN
3,africa eastern and southern,1999,NaN
4,africa eastern and southern,2000,3.500037
...,...,...,...
6645,zimbabwe,2016,5.800000
6646,zimbabwe,2017,6.000000
6647,zimbabwe,2018,6.400000
6648,zimbabwe,2019,6.700000


In [102]:
Indicators = pd.merge(cooking_data, rural_cooking_data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking
0,africa eastern and southern,1996,NaN,NaN
1,africa eastern and southern,1997,NaN,NaN
2,africa eastern and southern,1998,NaN,NaN
3,africa eastern and southern,1999,NaN,NaN
4,africa eastern and southern,2000,11.435351,3.500037
...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000
6646,zimbabwe,2017,29.800000,6.000000
6647,zimbabwe,2018,29.900000,6.400000
6648,zimbabwe,2019,30.100000,6.700000


In [103]:
temp = eg_indicators[2]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'urban_cooking'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking
0,africa eastern and southern,1996,NaN,NaN,NaN
1,africa eastern and southern,1997,NaN,NaN,NaN
2,africa eastern and southern,1998,NaN,NaN,NaN
3,africa eastern and southern,1999,NaN,NaN,NaN
4,africa eastern and southern,2000,11.435351,3.500037,32.330196
...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000
6646,zimbabwe,2017,29.800000,6.000000,79.300000
6647,zimbabwe,2018,29.900000,6.400000,79.100000
6648,zimbabwe,2019,30.100000,6.700000,79.100000


In [104]:
temp = eg_indicators[3]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
electricity = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'electricy'])
Indicators = pd.merge(Indicators, electricity, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283
...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475


In [105]:
temp = eg_indicators[4]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
rural_electricity = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'rural_electricy'])
Indicators = pd.merge(Indicators, rural_electricity, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418
...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877


In [106]:
temp = eg_indicators[5]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
urban_electricity = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'urban_electricy'])
Indicators = pd.merge(Indicators, urban_electricity, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196
...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336


In [107]:
temp = eg_indicators[6]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'nuclear_energy'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667
...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN


In [108]:
temp = eg_indicators[7]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'combustiable_renewable_waste'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701
...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN


In [109]:
temp = eg_indicators[8]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'electric_consumption'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,819.860432
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,829.825976
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,791.446996
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,769.795474
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,780.702624
...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,NaN


In [110]:
temp = eg_indicators[9]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'eletric_loss'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,819.860432,8.029657
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,829.825976,8.303374
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,791.446996,8.843838
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,769.795474,8.834762
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,780.702624,8.887529
...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,NaN,NaN


In [111]:
temp = eg_indicators[10]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'coal'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,819.860432,8.029657,75.208376
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,829.825976,8.303374,74.985428
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,791.446996,8.843838,72.659438
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,769.795474,8.834762,72.100508
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,780.702624,8.887529,71.504193
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,NaN,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,NaN,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,NaN,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,NaN,NaN,NaN


In [112]:
temp = eg_indicators[11]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'hydroeletric'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,819.860432,8.029657,75.208376,10.441763
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,829.825976,8.303374,74.985428,10.182238
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,791.446996,8.843838,72.659438,12.294936
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,769.795474,8.834762,72.100508,12.688264
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,780.702624,8.887529,71.504193,13.086301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,NaN,NaN,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,NaN,NaN,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,NaN,NaN,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
temp = eg_indicators[12]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'natrual_gas'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,819.860432,8.029657,75.208376,10.441763,0.000391
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,829.825976,8.303374,74.985428,10.182238,0.000754
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,791.446996,8.843838,72.659438,12.294936,0.000751
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,769.795474,8.834762,72.100508,12.688264,0.000750
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,780.702624,8.887529,71.504193,13.086301,0.000720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
temp = eg_indicators[13]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'nuclear'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,819.860432,8.029657,75.208376,10.441763,0.000391,4.603900
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,829.825976,8.303374,74.985428,10.182238,0.000754,4.768023
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,791.446996,8.843838,72.659438,12.294936,0.000751,5.108086
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,769.795474,8.834762,72.100508,12.688264,0.000750,4.816205
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,780.702624,8.887529,71.504193,13.086301,0.000720,4.681261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
temp = eg_indicators[14]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'oil'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,819.860432,8.029657,75.208376,10.441763,0.000391,4.603900,1.190172
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,829.825976,8.303374,74.985428,10.182238,0.000754,4.768023,1.487297
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,791.446996,8.843838,72.659438,12.294936,0.000751,5.108086,1.284815
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,769.795474,8.834762,72.100508,12.688264,0.000750,4.816205,1.768230
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,780.702624,8.887529,71.504193,13.086301,0.000720,4.681261,2.016792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
temp = eg_indicators[15]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'oil_gas_coal'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,819.860432,8.029657,75.208376,10.441763,0.000391,4.603900,1.190172,76.398939
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,829.825976,8.303374,74.985428,10.182238,0.000754,4.768023,1.487297,76.473480
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,791.446996,8.843838,72.659438,12.294936,0.000751,5.108086,1.284815,73.945003
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,769.795474,8.834762,72.100508,12.688264,0.000750,4.816205,1.768230,73.869489
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,780.702624,8.887529,71.504193,13.086301,0.000720,4.681261,2.016792,73.521705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
temp = eg_indicators[16]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'electric_from_renuable'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,819.860432,8.029657,75.208376,10.441763,0.000391,4.603900,1.190172,76.398939,0.312792
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,829.825976,8.303374,74.985428,10.182238,0.000754,4.768023,1.487297,76.473480,0.333652
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,791.446996,8.843838,72.659438,12.294936,0.000751,5.108086,1.284815,73.945003,0.409368
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,769.795474,8.834762,72.100508,12.688264,0.000750,4.816205,1.768230,73.869489,0.383436
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,780.702624,8.887529,71.504193,13.086301,0.000720,4.681261,2.016792,73.521705,0.468126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
temp = eg_indicators[17]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'electric_from_renuable_kwh'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,electric_consumption,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,819.860432,8.029657,75.208376,10.441763,0.000391,4.603900,1.190172,76.398939,0.312792,8.000000e+08
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,829.825976,8.303374,74.985428,10.182238,0.000754,4.768023,1.487297,76.473480,0.333652,8.850000e+08
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,791.446996,8.843838,72.659438,12.294936,0.000751,5.108086,1.284815,73.945003,0.409368,1.090000e+09
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,769.795474,8.834762,72.100508,12.688264,0.000750,4.816205,1.768230,73.869489,0.383436,1.022000e+09
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,780.702624,8.887529,71.504193,13.086301,0.000720,4.681261,2.016792,73.521705,0.468126,1.301000e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
temp = eg_indicators[18]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'energy_import'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,...,eletric_loss,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,...,8.029657,75.208376,10.441763,0.000391,4.603900,1.190172,76.398939,0.312792,8.000000e+08,-24.700335
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,...,8.303374,74.985428,10.182238,0.000754,4.768023,1.487297,76.473480,0.333652,8.850000e+08,-27.250868
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,...,8.843838,72.659438,12.294936,0.000751,5.108086,1.284815,73.945003,0.409368,1.090000e+09,-28.902742
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,...,8.834762,72.100508,12.688264,0.000750,4.816205,1.768230,73.869489,0.383436,1.022000e+09,-28.443490
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,...,8.887529,71.504193,13.086301,0.000720,4.681261,2.016792,73.521705,0.468126,1.301000e+09,-31.391070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
temp = eg_indicators[19]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'energy_intensity'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,...,coal,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,...,75.208376,10.441763,0.000391,4.603900,1.190172,76.398939,0.312792,8.000000e+08,-24.700335,NaN
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,...,74.985428,10.182238,0.000754,4.768023,1.487297,76.473480,0.333652,8.850000e+08,-27.250868,NaN
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,...,72.659438,12.294936,0.000751,5.108086,1.284815,73.945003,0.409368,1.090000e+09,-28.902742,NaN
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,...,72.100508,12.688264,0.000750,4.816205,1.768230,73.869489,0.383436,1.022000e+09,-28.443490,NaN
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,...,71.504193,13.086301,0.000720,4.681261,2.016792,73.521705,0.468126,1.301000e+09,-31.391070,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.00
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.51
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.83
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.47


In [121]:
temp = eg_indicators[20]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'energy_use(kg/capita)'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,...,hydroeletric,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita)
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,...,10.441763,0.000391,4.603900,1.190172,76.398939,0.312792,8.000000e+08,-24.700335,NaN,757.074853
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,...,10.182238,0.000754,4.768023,1.487297,76.473480,0.333652,8.850000e+08,-27.250868,NaN,755.589047
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,...,12.294936,0.000751,5.108086,1.284815,73.945003,0.409368,1.090000e+09,-28.902742,NaN,744.862070
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,...,12.688264,0.000750,4.816205,1.768230,73.869489,0.383436,1.022000e+09,-28.443490,NaN,746.661188
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,...,13.086301,0.000720,4.681261,2.016792,73.521705,0.468126,1.301000e+09,-31.391070,NaN,733.421542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.00,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.51,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.83,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.47,NaN


In [122]:
temp = eg_indicators[21]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'energy_use(kg/kGDP)'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,...,natrual_gas,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP)
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,...,0.000391,4.603900,1.190172,76.398939,0.312792,8.000000e+08,-24.700335,NaN,757.074853,231.526903
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,...,0.000754,4.768023,1.487297,76.473480,0.333652,8.850000e+08,-27.250868,NaN,755.589047,227.207711
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,...,0.000751,5.108086,1.284815,73.945003,0.409368,1.090000e+09,-28.902742,NaN,744.862070,226.053319
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,...,0.000750,4.816205,1.768230,73.869489,0.383436,1.022000e+09,-28.443490,NaN,746.661188,226.204585
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,...,0.000720,4.681261,2.016792,73.521705,0.468126,1.301000e+09,-31.391070,NaN,733.421542,220.091356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.00,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.51,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.83,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.47,NaN,NaN


In [123]:
temp = eg_indicators[22]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'fossil_fual_comsumption'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,...,nuclear,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,...,4.603900,1.190172,76.398939,0.312792,8.000000e+08,-24.700335,NaN,757.074853,231.526903,48.656467
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,...,4.768023,1.487297,76.473480,0.333652,8.850000e+08,-27.250868,NaN,755.589047,227.207711,48.608440
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,...,5.108086,1.284815,73.945003,0.409368,1.090000e+09,-28.902742,NaN,744.862070,226.053319,47.733478
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,...,4.816205,1.768230,73.869489,0.383436,1.022000e+09,-28.443490,NaN,746.661188,226.204585,47.928472
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,...,4.681261,2.016792,73.521705,0.468126,1.301000e+09,-31.391070,NaN,733.421542,220.091356,46.947252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.00,NaN,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9.51,NaN,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9.83,NaN,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.47,NaN,NaN,NaN


In [124]:
temp = eg_indicators[23]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'GDP/Energy(constrant_2017)'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,...,oil,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017)
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,...,1.190172,76.398939,0.312792,8.000000e+08,-24.700335,NaN,757.074853,231.526903,48.656467,4.319152
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,...,1.487297,76.473480,0.333652,8.850000e+08,-27.250868,NaN,755.589047,227.207711,48.608440,4.401259
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,...,1.284815,73.945003,0.409368,1.090000e+09,-28.902742,NaN,744.862070,226.053319,47.733478,4.423735
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,...,1.768230,73.869489,0.383436,1.022000e+09,-28.443490,NaN,746.661188,226.204585,47.928472,4.420777
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,...,2.016792,73.521705,0.468126,1.301000e+09,-31.391070,NaN,733.421542,220.091356,46.947252,4.543568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.00,NaN,NaN,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,9.51,NaN,NaN,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,9.83,NaN,NaN,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.47,NaN,NaN,NaN,NaN


In [125]:
temp = eg_indicators[24]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'GDP/Energy'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,...,oil_gas_coal,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017),GDP/Energy
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,...,76.398939,0.312792,8.000000e+08,-24.700335,NaN,757.074853,231.526903,48.656467,4.319152,2.944561
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,...,76.473480,0.333652,8.850000e+08,-27.250868,NaN,755.589047,227.207711,48.608440,4.401259,3.053433
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,...,73.945003,0.409368,1.090000e+09,-28.902742,NaN,744.862070,226.053319,47.733478,4.423735,3.107788
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,...,73.869489,0.383436,1.022000e+09,-28.443490,NaN,746.661188,226.204585,47.928472,4.420777,3.150793
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,...,73.521705,0.468126,1.301000e+09,-31.391070,NaN,733.421542,220.091356,46.947252,4.543568,3.313762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,...,NaN,NaN,NaN,NaN,10.00,NaN,NaN,NaN,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,...,NaN,NaN,NaN,NaN,9.51,NaN,NaN,NaN,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,...,NaN,NaN,NaN,NaN,9.83,NaN,NaN,NaN,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,...,NaN,NaN,NaN,NaN,10.47,NaN,NaN,NaN,NaN,NaN


In [126]:
temp = eg_indicators[25]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'renewable_output'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,...,electric_from_renuable,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017),GDP/Energy,renewable_output
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,...,0.312792,8.000000e+08,-24.700335,NaN,757.074853,231.526903,48.656467,4.319152,2.944561,12.738393
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,...,0.333652,8.850000e+08,-27.250868,NaN,755.589047,227.207711,48.608440,4.401259,3.053433,12.516603
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,...,0.409368,1.090000e+09,-28.902742,NaN,744.862070,226.053319,47.733478,4.423735,3.107788,14.875592
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,...,0.383436,1.022000e+09,-28.443490,NaN,746.661188,226.204585,47.928472,4.420777,3.150793,15.435672
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,...,0.468126,1.301000e+09,-31.391070,NaN,733.421542,220.091356,46.947252,4.543568,3.313762,15.983738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,...,NaN,NaN,NaN,10.00,NaN,NaN,NaN,NaN,NaN,NaN
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,...,NaN,NaN,NaN,9.51,NaN,NaN,NaN,NaN,NaN,NaN
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,...,NaN,NaN,NaN,9.83,NaN,NaN,NaN,NaN,NaN,NaN
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,...,NaN,NaN,NaN,10.47,NaN,NaN,NaN,NaN,NaN,NaN


In [127]:
temp = eg_indicators[26]
ind1 = eg_wdi[eg_wdi['Indicator Name'] == temp] #get data only for that indicator
ind1 = ind1.filter(['Country Name', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], axis=1)
#set country name to row index label
ind1 = ind1.set_index('Country Name') #uncomment later when want to run full

#country
countries = ind1.index.tolist() #countries becomes row index
country_ls = []
    
for i in countries:
    for j in range(len(ind1.columns)): #is 22 b/c 22 years of data from 1996 to 2018 len(ind1.columns) == 22
        country_ls.append(i)
        
# len(country_ls) #1197

#years
# last_yr = int(ind1.columns.tolist()[-1])
# first_yr = int(ind1.columns.tolist()[1])
# num_years = last_yr-first_yr
yrs = []
for i in ind1.columns:
    yrs.append(i[:]) #gets all the years in a list

yr_ls = []
range_val = int(len(country_ls)/len(ind1.columns))
for j in range(range_val): #len(country_ls)/19 : 19 is no. of years in time period
    yr_ls.append(yrs)
    
xls = []
for m in yr_ls:
    my_val = yr_ls[0]
    for n in my_val:
        xls.append(n) #takes years out of list of lists and just puts years together in order of 2000-2018 63 times
        
        
### DATA: clean_cook_pc

dt_ls = []
for i in range(range_val):
    vals = ind1.iloc[i].tolist()
    for j in vals:
        dt_ls.append(j)
        
##put in new tidy df
###create df
data = pd.DataFrame(list(zip(country_ls, xls,dt_ls)),
               columns =['Country Name', 'Year', 'renewable_comsumption'])
Indicators = pd.merge(Indicators, data, how='left', on=['Country Name','Year'])
Indicators

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,...,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017),GDP/Energy,renewable_output,renewable_comsumption
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,...,8.000000e+08,-24.700335,NaN,757.074853,231.526903,48.656467,4.319152,2.944561,12.738393,64.218668
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,...,8.850000e+08,-27.250868,NaN,755.589047,227.207711,48.608440,4.401259,3.053433,12.516603,63.385625
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,...,1.090000e+09,-28.902742,NaN,744.862070,226.053319,47.733478,4.423735,3.107788,14.875592,64.106365
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,...,1.022000e+09,-28.443490,NaN,746.661188,226.204585,47.928472,4.420777,3.150793,15.435672,65.168927
4,africa eastern and southern,2000,11.435351,3.500037,32.330196,19.811283,8.469418,53.563196,2.459667,39.727701,...,1.301000e+09,-31.391070,NaN,733.421542,220.091356,46.947252,4.543568,3.313762,15.983738,65.128796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,29.800000,5.800000,79.300000,42.561729,22.087053,85.483994,NaN,NaN,...,NaN,NaN,10.00,NaN,NaN,NaN,NaN,NaN,NaN,81.900000
6646,zimbabwe,2017,29.800000,6.000000,79.300000,44.178635,24.531088,85.478287,NaN,NaN,...,NaN,NaN,9.51,NaN,NaN,NaN,NaN,NaN,NaN,82.460000
6647,zimbabwe,2018,29.900000,6.400000,79.100000,45.572647,26.617121,85.468765,NaN,NaN,...,NaN,NaN,9.83,NaN,NaN,NaN,NaN,NaN,NaN,80.230000
6648,zimbabwe,2019,30.100000,6.700000,79.100000,46.781475,28.404877,85.457336,NaN,NaN,...,NaN,NaN,10.47,NaN,NaN,NaN,NaN,NaN,NaN,81.500000


In [128]:
Indicators.head(4)

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,...,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017),GDP/Energy,renewable_output,renewable_comsumption
0,africa eastern and southern,1996,NaN,NaN,NaN,NaN,NaN,NaN,2.292018,37.757141,...,8.000000e+08,-24.700335,NaN,757.074853,231.526903,48.656467,4.319152,2.944561,12.738393,64.218668
1,africa eastern and southern,1997,NaN,NaN,NaN,NaN,NaN,NaN,2.349575,37.792283,...,8.850000e+08,-27.250868,NaN,755.589047,227.207711,48.608440,4.401259,3.053433,12.516603,63.385625
2,africa eastern and southern,1998,NaN,NaN,NaN,NaN,NaN,NaN,2.451677,38.790478,...,1.090000e+09,-28.902742,NaN,744.862070,226.053319,47.733478,4.423735,3.107788,14.875592,64.106365
3,africa eastern and southern,1999,NaN,NaN,NaN,NaN,NaN,NaN,2.321523,38.932868,...,1.022000e+09,-28.443490,NaN,746.661188,226.204585,47.928472,4.420777,3.150793,15.435672,65.168927


In [129]:
Indicators.tail(4)

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,...,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017),GDP/Energy,renewable_output,renewable_comsumption
6646,zimbabwe,2017,29.8,6.0,79.3,44.178635,24.531088,85.478287,NaN,NaN,...,NaN,NaN,9.51,NaN,NaN,NaN,NaN,NaN,NaN,82.46
6647,zimbabwe,2018,29.9,6.4,79.1,45.572647,26.617121,85.468765,NaN,NaN,...,NaN,NaN,9.83,NaN,NaN,NaN,NaN,NaN,NaN,80.23
6648,zimbabwe,2019,30.1,6.7,79.1,46.781475,28.404877,85.457336,NaN,NaN,...,NaN,NaN,10.47,NaN,NaN,NaN,NaN,NaN,NaN,81.50
6649,zimbabwe,2020,30.4,6.8,78.6,52.747669,37.060249,85.715477,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
Indicators['Year'] = Indicators['Year'].astype(str)

In [131]:
Indicators.tail(4)

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,...,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017),GDP/Energy,renewable_output,renewable_comsumption
6646,zimbabwe,2017,29.8,6.0,79.3,44.178635,24.531088,85.478287,NaN,NaN,...,NaN,NaN,9.51,NaN,NaN,NaN,NaN,NaN,NaN,82.46
6647,zimbabwe,2018,29.9,6.4,79.1,45.572647,26.617121,85.468765,NaN,NaN,...,NaN,NaN,9.83,NaN,NaN,NaN,NaN,NaN,NaN,80.23
6648,zimbabwe,2019,30.1,6.7,79.1,46.781475,28.404877,85.457336,NaN,NaN,...,NaN,NaN,10.47,NaN,NaN,NaN,NaN,NaN,NaN,81.50
6649,zimbabwe,2020,30.4,6.8,78.6,52.747669,37.060249,85.715477,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Interpolation- Indicators

In [132]:
all_countries = list(np.unique(Indicators['Country Name'].tolist()))
all_years = list(np.unique(Indicators['Year'].tolist()))

In [133]:
cnt = 0
for i in all_countries:
    cnt += 1
    df = Indicators[Indicators['Country Name'] == i]
    country_yr = df[['Country Name','Year']]
    df.drop(['Country Name','Year'], axis=1, inplace=True)
    df.interpolate(method='linear', limit_direction='forward', limit_area='inside')
    
    df_new = pd.concat([country_yr, df], axis="columns")
    
    if cnt == 1:
        final_ind_df = df_new
    
    elif cnt >=2:
        final_ind_df = pd.concat([final_ind_df, df_new])

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/3032836189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [134]:
print('countries in final:',len(list(np.unique(final_ind_df['Country Name'].tolist()))))

print('countries in indicator:',len(all_countries))

countries in final: 266
countries in indicator: 266


In [135]:
final_ind_df.shape

(6650, 29)

In [136]:
Indicators.shape

(6650, 29)

In [137]:
#remove continents and regions from final_ind_df--only keep countries
remove = ['arab world', 'world','caribbean small states','central europe and the baltics','africa eastern and southern',
 'africa western and central','early-demographic dividend',
 'east asia & pacific',
 'east asia & pacific (excluding high income)',
 'east asia & pacific (ida & ibrd countries)','euro area',
 'europe & central asia',
 'europe & central asia (excluding high income)',
 'europe & central asia (ida & ibrd countries)',
 'european union','fragile and conflict affected situations','heavily indebted poor countries (hipc)','high income',
'ida & ibrd total',
 'ida blend',
 'ida only',
 'ida total', 'late-demographic dividend',
 'latin america & caribbean',
 'latin america & caribbean (excluding high income)',
 'latin america & the caribbean (ida & ibrd countries)','least developed countries: un classification','low & middle income',
 'low income',
 'lower middle income','middle east & north africa',
 'middle east & north africa (excluding high income)',
 'middle east & north africa (ida & ibrd countries)',
 'middle income','north america','post-demographic dividend',
 'pre-demographic dividend','pacific island small states','small states','south asia',
 'south asia (ida & ibrd)','sub-saharan africa',
 'sub-saharan africa (excluding high income)',
 'sub-saharan africa (ida & ibrd countries)',
         'not classified',
 'oecd members',
 'other small states','upper middle income']

final_ind_df = final_ind_df[~final_ind_df['Country Name'].isin(remove)]

In [138]:
all_countries = list(np.unique(final_ind_df['Country Name'].tolist()))
all_years = list(np.unique(final_ind_df['Year'].tolist()))

In [99]:
final_ind_df.shape

(5450, 29)

### % change calculation on interpolated data

In [139]:
# country_yr = final_ind_df[['Country Name', 'Year']]
# final_ind_df.drop(['Country Name', 'Year'],axis=1,inplace=True)
# pc_change_df = final_ind_df.pct_change() #gives (new value-old value)/old value
# pc_change_df = pc_change_df.multiply(100, axis="index") #multiplies change ratio by 100 to get percent
# pc_change_df = pd.concat([country_yr, pc_change_df],axis='columns')

In [140]:
cnt = 0
for i in all_countries:
    cnt += 1
    df = final_ind_df[final_ind_df['Country Name'] == i]
    country_yr = df[['Country Name','Year']]
    df.drop(['Country Name','Year'], axis=1, inplace=True)
    
    df = df.pct_change() #gives (new value-old value)/old value
    df = df.multiply(100, axis="index") #multiplies change ratio by 100 to get percent
    
    df_new = pd.concat([country_yr, df], axis="columns")
    
    if cnt == 1:
        pc_change_df = df_new
    
    elif cnt >=2:
        pc_change_df = pd.concat([pc_change_df, df_new])

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2897285658.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2897285658.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2897285658.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2897285658.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2897285658.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2897285658.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

### log change

In [141]:
cnt = 0
for i in all_countries:
    cnt += 1
    df = final_ind_df[final_ind_df['Country Name'] == i]
    country_yr = df[['Country Name','Year']]
    df.drop(['Country Name','Year'], axis=1, inplace=True)
    
    df = np.log(df)
    df = df.diff()
#     df = df.pct_change() #gives (new value-old value)/old value
#     df = df.multiply(100, axis="index") #multiplies change ratio by 100 to get percent
    
    df_new = pd.concat([country_yr, df], axis="columns")
    
    if cnt == 1:
        log_diff_df = df_new
    
    elif cnt >=2:
        log_diff_df = pd.concat([log_diff_df, df_new])

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2493170385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2493170385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2493170385.py:6: SettingWithC

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2493170385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2493170385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2493170385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2493170385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2493170385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2493170385.py:6: SettingWithC

/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2493170385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/var/folders/1c/ydvf19rs0v5dqfbdwntxk2080000gn/T/ipykernel_9298/2493170385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Country Name','Year'], axis=1, inplace=True)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarnin

In [142]:
log_diff_df

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,...,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017),GDP/Energy,renewable_output,renewable_comsumption
1225,afghanistan,1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1226,afghanistan,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.028269,0.087911
1227,afghanistan,1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027900,0.062263
1228,afghanistan,1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.009631,0.168479
1229,afghanistan,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.017033,-0.148370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,zimbabwe,2016,0.010118,0.053110,-0.003776,0.233458,0.701715,0.051414,NaN,NaN,...,NaN,NaN,-0.035367,NaN,NaN,NaN,NaN,NaN,NaN,0.013275
6646,zimbabwe,2017,0.000000,0.033902,0.000000,0.037286,0.104950,-0.000067,NaN,NaN,...,NaN,NaN,-0.050241,NaN,NaN,NaN,NaN,NaN,NaN,0.006814
6647,zimbabwe,2018,0.003350,0.064539,-0.002525,0.031066,0.081613,-0.000111,NaN,NaN,...,NaN,NaN,0.033095,NaN,NaN,NaN,NaN,NaN,NaN,-0.027416
6648,zimbabwe,2019,0.006667,0.045810,0.000000,0.026180,0.065006,-0.000134,NaN,NaN,...,NaN,NaN,0.063075,NaN,NaN,NaN,NaN,NaN,NaN,0.015706


In [143]:
pc_change_df.shape

(5450, 29)

In [144]:
sdg7_new = pd.DataFrame(data=sdg7_new)
sdg7_new.rename(columns={'PY': 'Year'}, inplace=True)
sdg7_new['Year'] = sdg7_new['Year'].astype(str)
sdg7_new

,Year,2 years window,ar index bulk
0,1991,6.0,0.875595
1,1992,2.0,0.454859
2,1993,4.0,0.823754
3,1994,6.0,1.122167
4,1995,3.0,1.850156
5,1996,13.0,2.734959
6,1997,19.0,3.201562
7,1998,29.0,4.313780
8,1999,46.0,6.392467
9,2000,62.0,11.032420


In [145]:
final = pd.merge(sdg7_new, pc_change_df,  how='left', on=['Year'])
final.to_csv('data/pc_change_citations_bulk_df.csv')

In [146]:
final = pd.merge(sdg7_new, log_diff_df,  how='left', on=['Year'])
final.to_csv('data/log_diff_citations_bulk_df.csv')

### Calculate Distance Measure on all countries and only OECD countries interpolated data

In [147]:
# drop country name that doesn't make sense: 'world', continent, regions etc

In [148]:
# all_years
inds = final_ind_df.columns.tolist()[2:]

In [149]:
inds

['cooking',
 'rural_cooking',
 'urban_cooking',
 'electricy',
 'rural_electricy',
 'urban_electricy',
 'nuclear_energy',
 'combustiable_renewable_waste',
 'electric_consumption',
 'eletric_loss',
 'coal',
 'hydroeletric',
 'natrual_gas',
 'nuclear',
 'oil',
 'oil_gas_coal',
 'electric_from_renuable',
 'electric_from_renuable_kwh',
 'energy_import',
 'energy_intensity',
 'energy_use(kg/capita)',
 'energy_use(kg/kGDP)',
 'fossil_fual_comsumption',
 'GDP/Energy(constrant_2017)',
 'GDP/Energy',
 'renewable_output',
 'renewable_comsumption']

In [150]:
list(np.unique(eg_wdi['Indicator Name'].tolist()))

['Access to clean fuels and technologies for cooking (% of population)',
 'Access to clean fuels and technologies for cooking, rural (% of rural population)',
 'Access to clean fuels and technologies for cooking, urban (% of urban population)',
 'Access to electricity (% of population)',
 'Access to electricity, rural (% of rural population)',
 'Access to electricity, urban (% of urban population)',
 'Alternative and nuclear energy (% of total energy use)',
 'Combustible renewables and waste (% of total energy)',
 'Electric power consumption (kWh per capita)',
 'Electric power transmission and distribution losses (% of output)',
 'Electricity production from coal sources (% of total)',
 'Electricity production from hydroelectric sources (% of total)',
 'Electricity production from natural gas sources (% of total)',
 'Electricity production from nuclear sources (% of total)',
 'Electricity production from oil sources (% of total)',
 'Electricity production from oil, gas and coal sources

In [151]:
# target_dict = {'electricy':100, 'cooking':97.50,'renewable_comsumption':25.18, 'renewable_output':38.65,
#                'energy_use(kg/capita)':18949.33, 'rural_cooking':100,'urban_cooking':100,
#                'rural_electricy':100,'urban_electricy':100, 'fossil_fual_comsumption':0, 'coal':0, 
#                'oil_gas_coal':0}
##############################################

# Renewable energy share in the total final energy consumption: (25.18)
# 'renewable_comsumption' (was previously put wrong-'electric_from_renuable')  

# Renewable electricity share in total electricity generation: (38.65)
# 'electric_from_renuable' OR
# 'renewable_output'

#Energy intensity level of primary energy ---> Energy Productivity (18949.33)
# 'energy_intensity', ---not this because overall value is < 10
#  'energy_use(kg/capita)',
#  'energy_use(kg/kGDP)', --not this
#  'GDP/Energy(constrant_2017)', --not this
#  'GDP/Energy', --not this

In [152]:
final_ind_df.shape

(5450, 29)

In [153]:
final_ind_df.head(5)

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,nuclear_energy,combustiable_renewable_waste,...,electric_from_renuable_kwh,energy_import,energy_intensity,energy_use(kg/capita),energy_use(kg/kGDP),fossil_fual_comsumption,GDP/Energy(constrant_2017),GDP/Energy,renewable_output,renewable_comsumption
1225,afghanistan,1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.370370,37.945748
1226,afghanistan,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.388060,41.432601
1227,afghanistan,1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.436090,44.094337
1228,afghanistan,1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.722628,52.185774
1229,afghanistan,2000,6.2,1.0,28.8,1.613591,NaN,72.133469,NaN,NaN,...,NaN,NaN,1.64,NaN,NaN,NaN,NaN,NaN,74.989094,44.990000


In [96]:
target_dict = {'electricy':100, 'cooking':97.50,'renewable_comsumption':25.18, 'renewable_output':38.65,
               'energy_use(kg/capita)':18949.33, 'rural_cooking':100,'urban_cooking':100,
               'rural_electricy':100,'urban_electricy':100, 'fossil_fual_comsumption':0, 'coal':0, 
               'oil_gas_coal':0}

dist_calc_ls = []
yr_ls = []
country_ls = []
# ind_ls = []

count = 0
for ind in inds:
    if ind in target_dict:
        count += 1
        ind_df = final_ind_df[['Country Name', 'Year', ind]] #df by indicator

        for yr in all_years:
            yr_df = ind_df[ind_df.Year == yr] #df by year for that indicator
            
            SD = np.std(yr_df[ind]) #standard dev for all countries in that year for this indicator
            
#             N = len(list(np.unique((yr_df['Country Name']).tolist()))) #total countries in that year for thos indicator
            
            for country in all_countries:
                country_val = list(yr_df[yr_df['Country Name'] == country][ind])[0] #gets the value for that ind, in that yr, for that country
                
                if SD == 0: #to avoid divsion by zero error
                    dist_calc = float("nan") #assign dist_calc as nan

                else:
                    calc = (target_dict[ind]-country_val)/SD #distance of country from OECD target divided by standard dev
                    dist_calc = (max(calc,0)) #max between calc and 0, multiplied by 1/N
                
                #get all values for this indicator
                dist_calc_ls.append(dist_calc)
                yr_ls.append(yr)
                country_ls.append(country)
        
        #put list of values in dataframe
        add_df = pd.DataFrame(list(zip(country_ls, yr_ls,dist_calc_ls)),
                   columns =['Country Name', 'Year', ind])

        if count == 1: #if it's the first dataframe, final df is add_df
            dist_df = add_df

        elif count >= 2: #when a dataframe exists, merge new indicator calcs to that df
            dist_df = (pd.merge(dist_df, add_df, how='left', on=['Country Name','Year']))

        #assign lists to empty list to get new values for next ind
        dist_calc_ls = []
        country_ls = []
        yr_ls = []
        

In [156]:
#for oecd only
oecd = ['australia','austria','belgium','canada','chile','czechia','denmark','estonia','finland','france',
'germany','greece','hungary','iceland','ireland','israel','italy','japan','portugal','slovak republic','slovenia',
'spain','sweden','switzerland','turkiye','united kingdom','united states', 'colombia', 'costa rica', 'korea, rep.',
        'latvia','lithuania',
 'luxembourg','mexico','netherlands',
 'new zealand','norway','poland']


#with oecd data
oecd_data = final_ind_df[final_ind_df['Country Name'].isin(oecd)]

target_dict = {'electricy':100, 'cooking':97.50,'renewable_comsumption':25.18, 'renewable_output':38.65,
               'energy_use(kg/capita)':18949.33, 'rural_cooking':100,'urban_cooking':100,
               'rural_electricy':100,'urban_electricy':100, 'fossil_fual_comsumption':0, 'coal':0, 
               'oil_gas_coal':0}

dist_calc_ls = []
yr_ls = []
country_ls = []
# ind_ls = []

count = 0
for ind in inds:
    if ind in target_dict:
        count += 1
        ind_df = oecd_data[['Country Name', 'Year', ind]] #df by indicator

        for yr in all_years:
            yr_df = ind_df[ind_df.Year == yr] #df by year for that indicator
            
            SD = np.std(yr_df[ind]) #standard dev for all countries in that year for this indicator
            
#             N = len(list(np.unique((yr_df['Country Name']).tolist()))) #total countries in that year for thos indicator
            
            for country in oecd:
#                 if country
                country_val = list(yr_df[yr_df['Country Name'] == country][ind])[0] #gets the value for that ind, in that yr, for that country
                
                if SD == 0: #to avoid divsion by zero error
                    dist_calc = float("nan") #assign dist_calc as nan

                else:
                    calc = (target_dict[ind]-country_val)/SD #distance of country from OECD target divided by standard dev
                    dist_calc = (max(calc,0)) #max between calc and 0, multiplied by 1/N
                
                #get all values for this indicator
                dist_calc_ls.append(dist_calc)
                yr_ls.append(yr)
                country_ls.append(country)
        
        #put list of values in dataframe
        add_df = pd.DataFrame(list(zip(country_ls, yr_ls,dist_calc_ls)),
                   columns =['Country Name', 'Year', ind])

        if count == 1: #if it's the first dataframe, final df is add_df
            dist_oecd_df = add_df

        elif count >= 2: #when a dataframe exists, merge new indicator calcs to that df
            dist_oecd_df = (pd.merge(dist_oecd_df, add_df, how='left', on=['Country Name','Year']))

        #assign lists to empty list to get new values for next ind
        dist_calc_ls = []
        country_ls = []
        yr_ls = []
        

In [157]:
dist_oecd_df.shape

(950, 14)

In [158]:
dist_oecd_df

,Country Name,Year,cooking,rural_cooking,urban_cooking,electricy,rural_electricy,urban_electricy,coal,oil_gas_coal,energy_use(kg/capita),fossil_fual_comsumption,renewable_output,renewable_comsumption
0,australia,1996,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,6.805797,0.0,0.949975,1.136048
1,austria,1996,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,7.716112,0.0,0.000000,0.079603
2,belgium,1996,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,6.734058,0.0,1.223893,1.670963
3,canada,1996,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.0,5.470381,0.0,0.000000,0.250744
4,chile,1996,NaN,NaN,NaN,3.076087,4.480422,2.532716,0.0,0.0,8.814472,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,mexico,2020,4.557182,4.637349,6.142359,6.189350,6.223856,6.028721,NaN,NaN,NaN,NaN,NaN,NaN
946,netherlands,2020,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
947,new zealand,2020,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
948,norway,2020,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
dist_df.sort_values(by=['Country Name', 'Year'])
# dist_df[dist_df['Country Name'] == 'zimbabwe']

,Country Name,Year,cooking,electricy,electric_from_renuable,energy_intensity
0,afghanistan,1996,NaN,NaN,NaN,NaN
218,afghanistan,1997,NaN,NaN,NaN,NaN
436,afghanistan,1998,NaN,NaN,NaN,NaN
654,afghanistan,1999,NaN,NaN,NaN,NaN
872,afghanistan,2000,2.266258,2.950515,NaN,4016.793634
...,...,...,...,...,...,...
4577,zimbabwe,2016,1.776870,2.226820,NaN,6399.540849
4795,zimbabwe,2017,1.780616,2.218425,NaN,6603.322339
5013,zimbabwe,2018,1.785873,2.240789,NaN,6688.645753
5231,zimbabwe,2019,1.788288,2.209658,NaN,6869.580567


In [ ]:
# print('len of dist calc', len(dist_calc_ls))
# print('len of yr calc', len(yr_ls))
# print('len of country calc', len(country_ls))
# print('len of ind calc', len(ind_ls))

#### Merge

In [97]:
sdg7_new = pd.DataFrame(data=sdg7_new)
sdg7_new.rename(columns={'PY': 'Year'}, inplace=True)
sdg7_new['Year'] = sdg7_new['Year'].astype(str)
sdg7_new

,Year,2 years window,ar index bulk
0,1996,4.0,1.232883
1,1997,1.0,1.645701
2,1998,15.0,3.329708
3,1999,8.0,3.417867
4,2000,15.0,5.200733
5,2001,41.0,6.591661
6,2002,103.0,12.504736
7,2003,69.0,5.859465
8,2004,89.0,7.627350
9,2005,111.0,9.562296


In [98]:
# final = pd.merge(sdg7_new, Indicators,  how='left', on=['Year'])
# final[final.Year == '2018']

##merge all dist_calc
final = pd.merge(sdg7_new, dist_df,  how='left', on=['Year'])
final.to_csv('data/dist_calc_citations_bulk_df.csv')

In [ ]:
# final.shape

In [ ]:
# final.tail(4)

In [ ]:
# final.info()

In [162]:
#merge for oecd
final = pd.merge(sdg7_new, dist_oecd_df,  how='left', on=['Year'])
final.to_csv('data/dist_calc_oecd_citations_bulk_df.csv')

### Calculate Simple Mean on all countries interpolated data

In [ ]:
cnt = 0
for yr in all_years:
    cnt+=1
    
    df = final_ind_df[final_ind_df.Year == yr]
    country_yr = df[['Country Name', 'Year']]
    df.drop(['Country Name','Year'], axis=1, inplace=True)
    
    df_norm = (df-df.min())/(df.max()-df.min()) # min-max scaling: min and max are across all countries that year.
    
    df_norm = pd.concat((country_yr, df_norm), axis='columns')
    
    if cnt == 1:
        final_norm_df = df_norm
    
    elif cnt >=2:
        final_norm_df = pd.concat([final_norm_df, df_norm])

In [ ]:
final_norm_df

In [ ]:
sdg7_new = pd.DataFrame(data=sdg7_new)
sdg7_new.rename(columns={'PY': 'Year'}, inplace=True)
sdg7_new['Year'] = sdg7_new['Year'].astype(str)
sdg7_new

In [ ]:
##merge all scaling/norm calc
final = pd.merge(sdg7_new, final_norm_df,  how='left', on=['Year'])
final.to_csv('data/norm_ind_citations_bulk_df.csv')

### Code safekeeping

#### Codes we don't need but may be useful

In [91]:
# for j in all_countries:
#     df = Indicators[Indicators['Country Name'] == j]
    
#     for i in df.columns:
#         if len(df) == df[i].isna().sum():
#             df.drop(i, axis=1, inplace=True)

#         else:
#             first_ind_df = df.index[0] #first index of the data frame
#             first_not_nan_ind = df[i].first_valid_index() #first non nan value's index in df--column specific
#             list_top_nans = list(np.arange(first_ind_df,first_not_nan_ind,1))

#             last_ind_df = df.index[-1] #last index of the data frame
#             last_not_nan_ind = df[i].last_valid_index() #last non nan value's index in df--column specific
#             list_bottom_nans = list(np.arange(last_not_nan_ind+1,last_ind_df+1,1))

#             if len(list_top_nans) != 0 and len(list_bottom_nans) != 0: #when there are nans at start and end
#                 df.drop(index=list(np.arange(first_ind_df,first_not_nan_ind,1)),inplace=True) #drop first few nans using nan indices
#                 df.drop(index=list(np.arange(last_not_nan_ind+1,last_ind_df+1,1)),inplace=True) #drop last few nans using nan indices

#             elif len(list_top_nans) == 0 and len(list_bottom_nans) != 0: #when only nans at end
#                 df.drop(index=list(np.arange(last_not_nan_ind+1,last_ind_df+1,1)),inplace=True) #drop last few nans using nan indices

#             #not working
#             elif len(list_top_nans) != 0 and len(list_bottom_nans) == 0: #when only nans at start
#                 df.drop(index=list(np.arange(first_ind_df,first_not_nan_ind,1)),inplace=True) #drop first few nans using nan indices
